#  "Explorando el corazón: Utilizando datos clínicos y de estilo de vida para predecir enfermedades cardíacas"

#### ENTRENAMIENTO DE LOS MODELOS

A lo largo de este notebook, abordaremos el problema de clasificación y nos enfocaremos en entrenar varios modelos con el objetivo de obtener las métricas más óptimas para nuestro problema. Nuestro objetivo final es encontrar el modelo que brinde el mejor rendimiento en la clasificación de los datos. A través de este proceso de entrenamiento y evaluación, exploraremos diferentes algoritmos y técnicas para optimizar nuestros modelos y lograr resultados confiables y eficientes. Estaremos atentos a métricas como la precisión, la exactitud, el puntaje F1 y otras medidas relevantes para evaluar el desempeño de cada modelo. Al final de este notebook, esperamos haber identificado el modelo más adecuado que se ajuste a nuestras necesidades y nos proporcione las mejores métricas en nuestra tarea de clasificación.

1. Logistic Regresion
2. Naive Bayes
3. Decision Tree Classifier
4. Random Forest Classifier
5. XGB Classifier




<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

In [59]:
# Importamos las distintas librerias necesarias para el análisis

# Tratamiento de datos
import numpy as np
import pandas as pd

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns

# Preprocesado y modelado
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,log_loss, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV

# Para guardar los modelos
import pickle

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

In [60]:
# Leemos el archivo de data/preprocessed_heart
df= pd.read_csv("../data/processed/processed_heart.csv")
df.head()

,HeartDisease,Smoking,Stroke,DiffWalking,Sex,AgeCategory_encoded,Diabetic_encoded,BMI_Category_Ordinal,GrupoSalud_Ordinal,GrupoSalud_Mental_Ordinal,SleepGroup_Ordinal
0,0,1,0,0,0,7,2,1,1,3,1
1,0,0,1,0,0,12,0,2,1,1,3
2,0,1,0,0,1,9,2,3,3,3,3
3,0,0,0,0,0,11,0,2,1,1,2
4,0,0,0,1,0,4,0,2,3,1,3


In [61]:
# Para observar mejor la correlación que se produce con respecto a la varible target de "HeartDisease"
# Se muestra en porcentage % y por orden ascendente
corr_matrix = df.corr()
corr_matrix['HeartDisease'].sort_values(ascending=False) *100

HeartDisease                 100.000000
AgeCategory_encoded           23.343224
DiffWalking                   20.125805
Stroke                        19.683530
Diabetic_encoded              16.855285
GrupoSalud_Ordinal            16.731876
Smoking                       10.776416
Sex                            7.004048
BMI_Category_Ordinal           5.342468
GrupoSalud_Mental_Ordinal      2.130066
SleepGroup_Ordinal            -3.456049
Name: HeartDisease, dtype: float64

In [62]:
# Usamos skew para calcular la asimetría de nuestro DataFrame
    # Un valor positivo indica una cola más larga hacia la derecha, lo que significa que hay valores extremos más altos. 
    # Un valor negativo indica una cola más larga hacia la izquierda, lo que significa que hay valores extremos más bajos.
df.skew(axis=0)

HeartDisease                 2.962525
Smoking                      0.355585
Stroke                       4.851460
DiffWalking                  2.088606
Sex                          0.099029
AgeCategory_encoded         -0.263611
Diabetic_encoded             2.088800
BMI_Category_Ordinal         0.659454
GrupoSalud_Ordinal           2.199359
GrupoSalud_Mental_Ordinal    1.808620
SleepGroup_Ordinal          -1.391516
dtype: float64

In [63]:
# Hacemos un bucle para comprobar todas las columnas cuales están desbalanceadas y cuales no
for column in df.columns:
    print(df[column].value_counts(sort=True))  # Contamos los valores de la columna y mostramos los resultados ordenados
    print()

HeartDisease
0    292422
1     27373
Name: count, dtype: int64

Smoking
0    187887
1    131908
Name: count, dtype: int64

Stroke
0    307726
1     12069
Name: count, dtype: int64

DiffWalking
0    275385
1     44410
Name: count, dtype: int64

Sex
0    167805
1    151990
Name: count, dtype: int64

AgeCategory_encoded
9     34151
8     33686
10    31065
7     29757
6     25382
12    24153
5     21791
11    21482
0     21064
4     21006
3     20550
2     18753
1     16955
Name: count, dtype: int64

Diabetic_encoded
0    269653
2     40802
1      6781
3      2559
Name: count, dtype: int64

BMI_Category_Ordinal
3    113640
2     88661
4     67157
5     27379
6     17844
1      5114
Name: count, dtype: int64

GrupoSalud_Ordinal
1    265043
2     28748
3     26004
Name: count, dtype: int64

GrupoSalud_Mental_Ordinal
1    247032
2     45891
3     26872
Name: count, dtype: int64

SleepGroup_Ordinal
3    222809
2     66721
1     30265
Name: count, dtype: int64



In [64]:
# Eliminamos las columnas que no necesitaremos más
df = df.drop(columns=(["SleepGroup_Ordinal", "GrupoSalud_Mental_Ordinal","BMI_Category_Ordinal", ]))


In [65]:
#definimos nuestras etiquetas y features
y = df['HeartDisease']
X = df.drop('HeartDisease', axis=1)

In [66]:
X.columns

Index(['Smoking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory_encoded',
       'Diabetic_encoded', 'GrupoSalud_Ordinal'],
      dtype='object')

In [67]:
# Dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [68]:
print("X_train", X_train.shape)
print("X_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

X_train (255836, 7)
X_test (63959, 7)
y_train (255836,)
y_test (63959,)


In [69]:
# Hay que dividir train y test y guardarlo en sus resptivas carpetas dentro de data

# Probamos el modelo sin aplicar el desbalanceo

In [70]:
# Escalar las características 

scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) # solo transform no fit(), ya que sino contaminariamos los datos 

In [71]:
logre = LogisticRegression(C=1.0,penalty='l2',random_state=42,solver="newton-cg")
logre.fit(X_train_scaled, y_train)

LogisticRegression(random_state=42, solver='newton-cg')

In [72]:
y_pred_Sin_Balanceo = logre.predict(X_test_scaled) 


In [73]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Suponiendo que ya has realizado todas las operaciones previas y tienes las variables adecuadas

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred_Sin_Balanceo)
print("Precisión del modelo:", accuracy)

# Calcular la precisión por clase
precision = precision_score(y_test, y_pred_Sin_Balanceo)
print("Precisión por clase:", precision)

# Calcular el recall (sensibilidad) por clase
recall = recall_score(y_test, y_pred_Sin_Balanceo)
print("Recall por clase:", recall)

# Calcular la puntuación F1 por clase
f1 = f1_score(y_test, y_pred_Sin_Balanceo)
print("Puntuación F1 por clase:", f1)

# Calcular el classification report
report = classification_report(y_test, y_pred_Sin_Balanceo)
print("Classification Report:")
print(report)


Precisión del modelo: 0.9125689895089042
Precisión por clase: 0.5
Recall por clase: 0.08369098712446352
Puntuación F1 por clase: 0.14338235294117646
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     58367
           1       0.50      0.08      0.14      5592

    accuracy                           0.91     63959
   macro avg       0.71      0.54      0.55     63959
weighted avg       0.88      0.91      0.88     63959





<details>
<summary>Explicación de las métricas</summary>
<p>
Precisión del modelo: 0.9137785467849363. Esta métrica representa la proporción de predicciones correctas sobre el total de predicciones realizadas. En este caso, el modelo tiene una precisión del 91.38%, lo que significa que aproximadamente el 91.38% de las predicciones son correctas.

Precisión por clase: 0.5062068965517241. Esta métrica representa la proporción de predicciones positivas que son correctas para la clase "1" (enfermedad cardíaca) en relación con todas las predicciones positivas realizadas para esa clase. Aquí, la precisión para la clase "1" es del 50.62%, lo que indica que alrededor del 50.62% de las predicciones positivas para la enfermedad cardíaca son correctas.

Recall por clase: 0.08854041013268998. Esta métrica, también conocida como sensibilidad o tasa de verdaderos positivos, representa la proporción de casos positivos (enfermedad cardíaca) que son correctamente identificados por el modelo. En este caso, el recall para la clase "1" es del 8.85%, lo que significa que el modelo identifica correctamente aproximadamente el 8.85% de los casos reales de enfermedad cardíaca.

Puntuación F1 por clase: 0.15071868583162218. La puntuación F1 es una medida que combina la precisión y el recall en una única métrica. Es útil cuando hay un desequilibrio entre las clases, como en este caso. La puntuación F1 para la clase "1" es del 15.07%, lo que indica un equilibrio entre la precisión y el recall para esa clase.

En cuanto al classification report, proporciona un resumen detallado de las métricas por clase y para el conjunto de datos completo. Observamos que la clase "0" (no enfermedad cardíaca) tiene una precisión alta, recall alto y puntuación F1 alta, mientras que la clase "1" (enfermedad cardíaca) muestra valores más bajos en todas las métricas. Esto puede indicar un desequilibrio en los datos, donde la clase "1" tiene menos representación en comparación con la clase "0". El conjunto de datos está sesgado hacia la clase "0" y eso puede afectar el rendimiento del modelo en la detección de la clase minoritaria (enfermedad cardíaca).

En resumen, las métricas indican que el modelo tiene una precisión general alta, pero tiene dificultades para identificar correctamente los casos de enfermedad cardíaca. Esto puede deberse al desequilibrio en los datos y es importante tenerlo en cuenta al interpretar los resultados y tomar decisiones basadas en ellos.

</p>
</details>

In [25]:
# Calcular la matriz de confusión 

confusion = confusion_matrix(y_test, y_pred_Sin_Balanceo) 
print("Matriz de confusión:") 
print(confusion) 

Matriz de confusión:
[[57899   468]
 [ 5124   468]]


In [26]:
# Calcular el classification report
report = classification_report(y_test, y_pred_Sin_Balanceo)
print("Classification Report:")
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.99      0.95     58367
           1       0.50      0.08      0.14      5592

    accuracy                           0.91     63959
   macro avg       0.71      0.54      0.55     63959
weighted avg       0.88      0.91      0.88     63959



In [74]:
modelo_SinBalanceo = y_pred_Sin_Balanceo
with open('../models/trained_model_Sin_Balanceo_Reg_Log.pkl', 'wb') as file:
    pickle.dump(modelo_SinBalanceo, file) 

# APLICACIÓN DE LAS DISTINTAS TÉCNICAS PARA DESBALANCEAR EL PROBLEMA
En esta sección, aplicaremos diferentes técnicas de desbalanceo de datos utilizando el mismo modelo con el objetivo de determinar la estrategia más efectiva. Evaluaremos las siguientes estrategias:

<details>
<summary>Técnicas utilizadas</summary>
<p>


**Penalización para compensar**: Esta estrategia implica la aplicación de penalizaciones en el algoritmo de aprendizaje automático para compensar el desequilibrio en los datos. Se le asigna un peso mayor a las instancias de la clase minoritaria durante el entrenamiento para asegurar que se les dé más importancia en la predicción final.

**Oversampling de la clase minoritaria**: En esta estrategia, generamos nuevas muestras sintéticas de la clase minoritaria para igualar la cantidad de muestras con la clase mayoritaria. Esto se logra replicando o generando nuevas instancias de la clase minoritaria en el conjunto de datos.

**Combinación de resampling con Smote-Tomek:** Esta estrategia combina el undersampling y el oversampling para abordar el desbalanceo. El undersampling se aplica para reducir la cantidad de muestras de la clase mayoritaria, mientras que el oversampling se utiliza para generar nuevas muestras sintéticas de la clase minoritaria. Smote-Tomek es una técnica específica que combina el algoritmo SMOTE (Synthetic Minority Over-sampling Technique) con el método Tomek, que elimina instancias cercanas entre las dos clases.

**Ensamble de Modelos con Balanceo**: En esta estrategia, utilizamos un ensamble de modelos para abordar el desbalanceo. Se entrenan varios modelos de aprendizaje automático utilizando diferentes técnicas de balanceo de datos. Luego, se combinan las predicciones de estos modelos para obtener una predicción final más robusta y equilibrada.

Al evaluar estas estrategias y comparar su desempeño, podremos determinar cuál es la más adecuada para abordar el desbalanceo en nuestros datos y mejorar la calidad de nuestro modelo de aprendizaje automático.








</p>
</details>

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
 
from pylab import rcParams
 
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
 
from collections import Counter

### Estrategia: Penalización para compensar

Utilizaremos un parámetro adicional en el modelo de Regresión logística en donde indicamos weight = “balanced” y con esto el algoritmo se encargará de equilibrar a la clase minoritaria durante el entrenamiento

In [59]:
#dividimos en sets de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# EscalaMOS las características 
scaler = StandardScaler() 
X_train_scaled = scaler.fit_transform(X_train) 
X_test_scaled = scaler.transform(X_test) # solo transform no fit(), ya que sino contaminariamos los datos 

# Modelo de Logistic Regression
logre2 = LogisticRegression(C=0.1,penalty='l1',random_state=42,solver="liblinear", class_weight= "balanced") # únicamente incorporamos el class_weight= "balanced"
logre2.fit(X_train_scaled, y_train)
y_pred2 = logre2.predict(X_test_scaled) 

In [60]:
# Calcular el classification report
report2 = classification_report(y_test, y_pred2)
print("Classification Report:")
print(report2)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.73      0.83     58367
           1       0.21      0.77      0.33      5592

    accuracy                           0.73     63959
   macro avg       0.59      0.75      0.58     63959
weighted avg       0.90      0.73      0.79     63959



<details>
<summary>Explicación de las métricas</summary>
<p>
En comparación con el modelo anterior, el nuevo modelo de Logistic Regression con los datos de prueba actualizados muestra una mejora en ciertas métricas, mientras que otras disminuyen. Aquí hay un resumen de las diferencias clave:

Precisión general: El nuevo modelo tiene una precisión general de 0.74, lo que indica que aproximadamente el 74% de las predicciones son correctas. Esta precisión es menor que la del modelo anterior, que era de 0.91.

Precisión por clase: Para la clase "0" (no enfermedad cardíaca), la precisión es alta con un valor de 0.97, lo que indica que la mayoría de las predicciones negativas son correctas. Sin embargo, para la clase "1" (enfermedad cardíaca), la precisión es baja con un valor de 0.22, lo que indica que la mayoría de las predicciones positivas son incorrectas.

Recall por clase: El nuevo modelo muestra un recall alto para la clase "1" (enfermedad cardíaca) con un valor de 0.77, lo que indica que la mayoría de los casos reales de enfermedad cardíaca son correctamente identificados por el modelo. Sin embargo, el recall para la clase "0" (no enfermedad cardíaca) es de 0.74, lo que indica que hay algunos casos reales de no enfermedad cardíaca que no son identificados por el modelo.

Puntuación F1 por clase: La puntuación F1 para la clase "1" (enfermedad cardíaca) es de 0.34, lo que indica un equilibrio entre la precisión y el recall para esa clase. Sin embargo, la puntuación F1 para la clase "0" (no enfermedad cardíaca) es de 0.84, lo que indica un mejor equilibrio entre precisión y recall para esa clase.

En general, el nuevo modelo muestra una mejora en la identificación de casos de enfermedad cardíaca, como se refleja en un recall más alto para la clase "1". Sin embargo, la precisión para la clase "1" ha disminuido, lo que significa que hay un mayor número de falsos positivos en comparación con el modelo anterior. Además, la precisión general y la puntuación F1 han disminuido. Estos resultados sugieren que el modelo todavía puede beneficiarse de ajustes o considerar técnicas adicionales para abordar el desequilibrio en los datos y mejorar la detección de enfermedades cardíacas.

</p>
</details>

In [36]:
# Calculamos la matriz de confusión
confusion2 = confusion_matrix(y_test, y_pred2)
print("Matriz de confusión:")
print(confusion2)




Matriz de confusión:
[[42480 15887]
 [ 1280  4312]]


### Estrategia: Oversampling de la clase minoritaria

En este caso, crearemos muestras nuevas “sintéticas” de la clase minoritaria. Usando RandomOverSampler.

In [57]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from collections import Counter

# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Aplicar RandomOverSampler
os = RandomOverSampler(sampling_strategy=0.5)
X_train_res, y_train_res = os.fit_resample(X_train_scaled, y_train)

# Imprimir distribución antes y después del resampling
print("Distribución antes del resampling:", Counter(y_train))
print("Distribución después del resampling:", Counter(y_train_res))

# Definir y entrenar el modelo
model = LogisticRegression(C=0.1,penalty='l2',random_state=42,solver="newton-cg", class_weight= "balanced") # únicamente incorporamos el class_weight= "balanced"
model.fit(X_train_res, y_train_res)

# Realizar predicciones en el conjunto de prueba
pred_y_random = model.predict(X_test_scaled)


Distribución antes del resampling: Counter({0: 234055, 1: 21781})
Distribución después del resampling: Counter({0: 234055, 1: 117027})


In [58]:
# Calcular el classification report
report4 = classification_report(y_test, pred_y_random)
print("Classification Report:")
print(report4)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.73      0.83     58367
           1       0.21      0.77      0.33      5592

    accuracy                           0.73     63959
   macro avg       0.59      0.75      0.58     63959
weighted avg       0.90      0.73      0.79     63959



In [236]:
# Calculamos la matriz de confusión
confusion4 = confusion_matrix(y_test, pred_y_random)
print("Matriz de confusión:")
print(confusion4)

Matriz de confusión:
[[42593 15774]
 [ 1302  4290]]


### Estrategia: Combinamos resampling con Smote-Tomek

Ahora probaremos una técnica muy usada que consiste en aplicar en simultáneo un algoritmo de subsampling y otro de oversampling a la vez al dataset. En este caso usaremos SMOTE para oversampling: busca puntos vecinos cercanos y agrega puntos “en linea recta” entre ellos. Y usaremos Tomek para undersampling que quita los de distinta clase que sean nearest neighbor y deja ver mejor el decisión boundary (la zona limítrofe de nuestras clases).

In [29]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from collections import Counter

# Aplicar SMOTE para oversampling
smote = SMOTE()
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Aplicar Tomek Links para undersampling
tomek = TomekLinks()
X_train_res, y_train_res = tomek.fit_resample(X_train_res, y_train_res)

print("Distribución antes del resampling:", Counter(y_train))
print("Distribución después del resampling:", Counter(y_train_res))

# Entrenar un modelo (reemplaza esta parte con tu propio código de entrenamiento de modelo)
model = LogisticRegression(C=1.0, penalty='l2', random_state=42, solver="newton-cg", class_weight="balanced")
model.fit(X_train_res, y_train_res)

# Realizar predicciones en el conjunto de prueba
pred_y = model.predict(X_test)





Distribución antes del resampling: Counter({0: 234055, 1: 21781})
Distribución después del resampling: Counter({0: 234055, 1: 233355})


In [30]:
# Calcular el classification report
report3 = classification_report(y_test, pred_y)
print("Classification Report:")
print(report3)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.70      0.81     58367
           1       0.20      0.78      0.31      5592

    accuracy                           0.70     63959
   macro avg       0.58      0.74      0.56     63959
weighted avg       0.90      0.70      0.77     63959



In [31]:
# Calculamos la matriz de confusión
confusion3 = confusion_matrix(y_test, pred_y)
print("Matriz de confusión:")
print(confusion3)

Matriz de confusión:
[[40667 17700]
 [ 1252  4340]]


### Estrategia: Ensamble de Modelos con Balanceo

Para esta estrategia usaremos un Clasificador de Ensamble que utiliza Bagging y el modelo será un DecisionTree

In [42]:
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='auto',
                                replacement=True,
                                random_state=0,)
 
#Train the classifier.
bbc.fit(X_train, y_train)
pred_yBagging = bbc.predict(X_test)


In [43]:
# Calcular el classification report
report4 = classification_report(y_test, pred_yBagging)
print("Classification Report:")
print(report4)

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.70      0.81     58367
           1       0.20      0.80      0.32      5592

    accuracy                           0.71     63959
   macro avg       0.59      0.75      0.57     63959
weighted avg       0.91      0.71      0.77     63959



In [44]:
# Calculamos la matriz de confusión
confusion4 = confusion_matrix(y_test, pred_yBagging)
print("Matriz de confusión:")
print(confusion4)

Matriz de confusión:
[[40827 17540]
 [ 1118  4474]]


### Resultados de las Estrategias


In [ ]:
import numpy as np

# Calcula los pesos de clase
total_samples = len(y)
class_weights = total_samples / (len(np.unique(y)) * np.bincount(y))

# Crea un diccionario de pesos de clase
class_weight_dict = {class_index: weight for class_index, weight in enumerate(class_weights)}

# Compila el modelo utilizando los pesos de clase
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['recall'], class_weight=class_weight_dict)

# Entrena el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


### MODELOS A TRATAR

A lo largo de este apartado realizaremos la solución de problema a través de diferentes modelos supervisados entre los que nos encontramos con..... en busca de las mejores métricas posibles para nuestro problema.......

### MODELO SUPERVISADO
<details>
<summary>¿Qué es el aprendizaje supervisado?</summary>
<p>


</p>
</details>

<details>
<summary>¿Qué modelos vamos a tratar?</summary>
<p>


</p>
</details>

#### LOGISTIC REGRESION

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest


In [75]:
# Creamos el pipeline con escalamiento y modelo Logistic Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selectkbest', SelectKBest()),
    ('logre2', LogisticRegression())
])

# Definimos los parámetros a probar en el GridSearchCV
parameters = {
    'logre2__C': [0.1, 1, 10], # regularización
    'logre2__penalty': ['l1', 'l2'], # Lasso y Ridge
    'logre2__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # se prueba cada algoritmo
    'logre2__class_weight': [None, 'balanced'],  # sin balanceo y con balanceo
    'selectkbest__k': [5, 6, 7] # se seleccionan los mejores k
}

In [77]:
# Realizamos la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='recall') # He probado con varios cv y el cv=5 es el que mejor se adapta
grid_search.fit(X_train, y_train)

# Obtenemos las predicciones del mejor modelo encontrado
y_pred_logRe = grid_search.predict(X_test)

In [78]:
# Calculamos el classification report
report2 = classification_report(y_test, y_pred_logRe)
print("Classification Report:")
print(report2)

# Mejores hiperparámetros encontrados
best_params_logRe = grid_search.best_params_
print("Mejores hiperparámetros:")
print(best_params_logRe)


Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.69      0.81     58367
           1       0.19      0.77      0.31      5592

    accuracy                           0.70     63959
   macro avg       0.58      0.73      0.56     63959
weighted avg       0.90      0.70      0.76     63959

Mejores hiperparámetros:
{'logre2__C': 0.1, 'logre2__class_weight': 'balanced', 'logre2__penalty': 'l1', 'logre2__solver': 'liblinear', 'selectkbest__k': 5}


In [79]:
mejor_modelo_logRe = y_pred_logRe
with open('../models/trained_model_Log_Regression.pkl', 'wb') as file:
    pickle.dump(mejor_modelo_logRe, file) 


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### NAIVE BAYES

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

In [49]:
# Creamos el pipeline con escalado, selección de características, SMOTE y modelo Naive Bayes
pipeline = Pipeline([
    ('selectkbest', SelectKBest()),
    ('smote', SMOTE()),  # Aplicar oversampling con SMOTE
    ('naive_bayes', GaussianNB())
])

# Definimos los parámetros a probar en el GridSearchCV
parameters = {
    'selectkbest__k': [7,8,9],
    'smote__sampling_strategy': [0.5, 0.75, 1.0],  # Ajustar la proporción de oversampling
    'naive_bayes__var_smoothing': [1e-9,1e-10,1e-11]  # Ajustar el suavizado de varianza 
}

In [50]:
# Realizamos la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=10, scoring='recall')
grid_search.fit(X_train, y_train)

# Obtenemos las predicciones del mejor modelo encontrado
y_pred_Naive = grid_search.predict(X_test)

ValueError: 
All the 270 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\alexm\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\alexm\AppData\Roaming\Python\Python310\site-packages\sklearn\pipeline.py", line 401, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "C:\Users\alexm\AppData\Roaming\Python\Python310\site-packages\sklearn\pipeline.py", line 339, in _fit
    self._validate_steps()
  File "C:\Users\alexm\AppData\Roaming\Python\Python310\site-packages\sklearn\pipeline.py", line 230, in _validate_steps
    raise TypeError(
TypeError: All intermedi

In [52]:
# Calculmosr el classification report
report3 = classification_report(y_test, y_pred_Naive)
print("Classification Report:")
print(report3)

# Mejores hiperparámetros encontrados
best_params_Naive = grid_search.best_params_
print("Mejores hiperparámetros:")
print(best_params_Naive)

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.80      0.87     58367
           1       0.23      0.60      0.33      5592

    accuracy                           0.79     63959
   macro avg       0.59      0.70      0.60     63959
weighted avg       0.89      0.79      0.82     63959

Mejores hiperparámetros:
{'naive_bayes__var_smoothing': 1e-10, 'selectkbest__k': 7, 'smote__sampling_strategy': 1.0}


In [ ]:
mejor_modelo_Naive = y_pred_Naive
with open('../models/trained_model_NaiveBayes.pkl', 'wb') as file:
    pickle.dump(mejor_modelo_Naive, file) 


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### SVC


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### DECISSION TREE CLASSIFIER

In [24]:
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# TARDA MUCHO NO EJECUTAR

# Crear el pipeline con SMOTE, BalancedBaggingClassifier y DecisionTreeClassifier
pipeline = Pipeline([
    ('bbc', BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                      random_state=0,
                                      sampling_strategy='auto'))
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'bbc__n_estimators': [50, 80, 100],
    'bbc__base_estimator__max_depth': [5, 8, 10],
    'bbc__base_estimator__min_samples_split': [2, 5, 10],
    'bbc__base_estimator__min_samples_leaf': [1, 2, 4],
    'bbc__base_estimator__class_weight': ['balanced', None]
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='recall')
grid_search.fit(X_train, y_train)

# Obtener las mejores configuraciones de hiperparámetros encontradas
best_params = grid_search.best_params_

# Obtener las predicciones del mejor modelo encontrado
y_pred = grid_search.predict(X_test)

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred))



KeyboardInterrupt: 

In [23]:
# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:")
print(best_params)

Mejores hiperparámetros:
{'bbc__base_estimator__max_depth': 10, 'bbc__n_estimators': 100}


In [ ]:
with open('../models/trained_model.pkl', 'wb') as file:
    pickle.dump(mejor_modelo, file) # reemplaza mejor_modelo con el nombre de la variable que contiene el mejor modelo entrenado.


In [25]:
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split


# Crear el pipeline con DecisionTreeClassifier
pipeline = Pipeline([
    ('dt', DecisionTreeClassifier(random_state=42, class_weight='balanced')) # Para solucionar el problema de balanceo class_weight='balanced'
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'dt__max_depth': [5, 8, 10],
    'dt__min_samples_split': [2, 5, 10],
    'dt__min_samples_leaf': [1, 2, 4]
}

# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search = GridSearchCV(pipeline, parameters, cv=5, scoring='recall')
grid_search.fit(X_train, y_train)

# Obtener las mejores configuraciones de hiperparámetros encontradas
best_params = grid_search.best_params_



In [ ]:
best_params

In [27]:
# Entrenar el modelo con los mejores hiperparámetros
pipeline.set_params(**best_params) # ** se utiliza para desempaquetar un diccionario y pasar sus elementos como argumentos de palabras clave a una función o método
pipeline.fit(X_train, y_train)

# Obtener las predicciones
y_pred_decisiontree = pipeline.predict(X_test)

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred_decisiontree))

              precision    recall  f1-score   support

           0       0.97      0.65      0.78     58367
           1       0.18      0.81      0.30      5592

    accuracy                           0.66     63959
   macro avg       0.58      0.73      0.54     63959
weighted avg       0.90      0.66      0.74     63959



In [28]:
import pickle


In [29]:
mejor_modelo = y_pred_decisiontree

In [30]:
with open('../models/trained_model_decisionTree.pkl', 'wb') as file:
    pickle.dump(mejor_modelo, file) # reemplaza mejor_modelo con el nombre de la variable que contiene el mejor modelo entrenado.



<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### RANDOM FOREST CLASSIFIER

In [31]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV

# Crear el pipeline con Random Forest Classifier
pipeline = Pipeline([
    ('rfc', RandomForestClassifier(random_state=0))
])

# Definir los parámetros a probar en el RandomizedSearchCV
parameters = {
    'rfc__n_estimators': [50, 80, 100],
    'rfc__max_depth': [5, 8, 10],
    'rfc__min_samples_split': [2, 5, 10],
    'rfc__min_samples_leaf': [1, 2, 4],
    'rfc__class_weight': ['balanced', None]
}

# Realizar la búsqueda aleatoria de hiperparámetros utilizando RandomizedSearchCV
random_search = RandomizedSearchCV(pipeline, parameters, cv=5, scoring='recall', random_state=0)
random_search.fit(X_train, y_train)


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('rfc',
                                              RandomForestClassifier(random_state=0))]),
                   param_distributions={'rfc__class_weight': ['balanced', None],
                                        'rfc__max_depth': [5, 8, 10],
                                        'rfc__min_samples_leaf': [1, 2, 4],
                                        'rfc__min_samples_split': [2, 5, 10],
                                        'rfc__n_estimators': [50, 80, 100]},
                   random_state=0, scoring='recall')

In [32]:
# Obtener las mejores configuraciones de hiperparámetros encontradas
best_params_RandomForest = random_search.best_params_
best_params_RandomForest

{'rfc__n_estimators': 100,
 'rfc__min_samples_split': 10,
 'rfc__min_samples_leaf': 2,
 'rfc__max_depth': 8,
 'rfc__class_weight': 'balanced'}

In [33]:
# Obtener las predicciones del mejor modelo encontrado
y_pred_RandomForest = random_search.predict(X_test)

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred_RandomForest))

              precision    recall  f1-score   support

           0       0.97      0.70      0.81     58367
           1       0.20      0.80      0.32      5592

    accuracy                           0.71     63959
   macro avg       0.59      0.75      0.57     63959
weighted avg       0.91      0.71      0.77     63959



In [35]:
mejor_modelo_RandomForest = y_pred_decisiontree
with open('../models/trained_model_RandomForest.pkl', 'wb') as file:
    pickle.dump(mejor_modelo_RandomForest, file) 


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### GAUSSIANNB


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### XGBCCLASSIFIER

In [80]:
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler


# Crear el pipeline con escalado y XGBoost Classifier con scale_pos_weight
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBClassifier(random_state=0, scale_pos_weight=3)) # scale_pos_weight=3 para el balanceo
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'xgb__max_depth': [2, 3, 4],
    'xgb__learning_rate': [0.1, 0.01, 0.001]
}



In [81]:
# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search_XGB = GridSearchCV(pipeline, parameters, cv=5, scoring='recall')
grid_search_XGB.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('xgb',
                                        XGBClassifier(base_score=None,
                                                      booster=None,
                                                      callbacks=None,
                                                      colsample_bylevel=None,
                                                      colsample_bynode=None,
                                                      colsample_bytree=None,
                                                      early_stopping_rounds=None,
                                                      enable_categorical=False,
                                                      eval_metric=None,
                                                      feature_types=None,
                                                      gamma=None, gpu_id=None,
                                                      grow_policy=None,
                                                      importance_type=No...
                                                      max_bin=None,
                                                      max_cat_threshold=None,
                                                      max_cat_to_onehot=None,
                                                      max_delta_step=None,
                                                      max_depth=None,
                                                      max_leaves=None,
                                                      min_child_weight=None,
                                                      missing=nan,
                                                      monotone_constraints=None,
                                                      n_estimators=100,
                                                      n_jobs=None,
                                                      num_parallel_tree=None,
                                                      predictor=None,
                                                      random_state=0, ...))]),
             param_grid={'xgb__learning_rate': [0.1, 0.01, 0.001],
                         'xgb__max_depth': [2, 3, 4]},
             scoring='recall')

In [82]:
# Obtener las mejores configuraciones de hiperparámetros encontradas
best_params_XGB = grid_search_XGB.best_params_
best_params_XGB

{'xgb__learning_rate': 0.1, 'xgb__max_depth': 4}

In [83]:
# Obtener las predicciones del mejor modelo encontrado
y_pred_XGB = grid_search_XGB.predict(X_test)

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred_XGB))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94     58367
           1       0.36      0.33      0.34      5592

    accuracy                           0.89     63959
   macro avg       0.65      0.64      0.64     63959
weighted avg       0.89      0.89      0.89     63959



In [48]:
mejor_modelo_XGB = y_pred_XGB
with open('../models/trained_model_XGB_Classifiert.pkl', 'wb') as file:
    pickle.dump(mejor_modelo_XGB, file) 


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

### MODELO NO SUPERVISADO
<details>
<summary>¿Qué es el aprendizaje no supervisado?</summary>
<p>


</p>
</details>

#### KNN CLASSIIFIER


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

#### PCA

In [42]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.linear_model import LogisticRegression

# Crear el pipeline con escalamiento, PCA, selección de características y modelo Logistic Regression
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('selectkbest', SelectKBest()),
    ('logre2', LogisticRegression())
])

# Definir los parámetros a probar en el GridSearchCV
parameters = {
    'pca__n_components': [2, 3, 4],  # Número de componentes para PCA
    'logre2__C': [0.1, 1, 10],  # Regularización
    'logre2__penalty': ['l1', 'l2'],  # Lasso y Ridge
    'logre2__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],  # Algoritmos de optimización
    'logre2__class_weight': [None, 'balanced'],  # Sin balanceo y con balanceo
    'selectkbest__k': [3, 4, 5]  # Seleccionar los mejores k
}


In [43]:
# Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
grid_search_pca = GridSearchCV(pipeline, parameters, cv=5, scoring='recall')
grid_search_pca.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('selectkbest', SelectKBest()),
                                       ('logre2', LogisticRegression())]),
             param_grid={'logre2__C': [0.1, 1, 10],
                         'logre2__class_weight': [None, 'balanced'],
                         'logre2__penalty': ['l1', 'l2'],
                         'logre2__solver': ['newton-cg', 'lbfgs', 'liblinear',
                                            'sag', 'saga'],
                         'pca__n_components': [2, 3, 4],
                         'selectkbest__k': [3, 4, 5]},
             scoring='recall')

In [44]:
# Obtener las mejores configuraciones de hiperparámetros encontradas
best_params_pca = grid_search.best_params_

# Obtener las predicciones del mejor modelo encontrado
y_pred_pca = grid_search_pca.predict(X_test)

In [45]:
# Obtener las predicciones del mejor modelo encontrado
y_pred_pca = grid_search_pca.predict(X_test)

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred_pca))

              precision    recall  f1-score   support

           0       0.96      0.75      0.84     58367
           1       0.21      0.71      0.33      5592

    accuracy                           0.75     63959
   macro avg       0.59      0.73      0.59     63959
weighted avg       0.90      0.75      0.80     63959



In [46]:
mejor_modelo_pca = y_pred_pca
with open('../models/trained_model_PCA.pkl', 'wb') as file:
    pickle.dump(mejor_modelo_pca, file) 


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>

### REDES NEURONALES

In [10]:
from sklearn.neural_network import MLPClassifier 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='sgd',
            loss = 'sparse_categorical_crossentropy',
            metrics = ['recall'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

In [ ]:
pd.DataFrame(history.history)['val_recall'].plot()


<details>
<summary>Explicación de las métricas</summary>
<p>


</p>
</details>